In [1]:
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import numpy as np
import re

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
}
Team_ID = []
Team_Name = []
season = 2000
while True:
  base_url = f"https://www.transfermarkt.com/egyptian-premier-league/startseite/wettbewerb/EGY1/plus/?saison_id={season}"
  pageTree = requests.get(base_url, headers=headers)
  soup = BeautifulSoup(pageTree.content, 'html.parser')
  teams = soup.find_all('td',{'class':'hauptlink no-border-links'})
  for team in teams:
    if team.find('a'):
      href = team.find('a')['href']
      team_id = href.split('/verein/')[1].split('/')[0]
      Team_ID.append(team_id)
      Team_Name.append(team.text.strip())
  season += 1
  print(f'Season {season} Scraped')
  # Stop after scraping up to season 2001
  if season > 2023:
        break
Teams_Data = dict(zip(Team_ID, Team_Name))
print(Teams_Data)

Season 2001 Scraped
Season 2002 Scraped
Season 2003 Scraped
Season 2004 Scraped
Season 2005 Scraped
Season 2006 Scraped
Season 2007 Scraped
Season 2008 Scraped
Season 2009 Scraped
Season 2010 Scraped
Season 2011 Scraped
Season 2012 Scraped
Season 2013 Scraped
Season 2014 Scraped
Season 2015 Scraped
Season 2016 Scraped
Season 2017 Scraped
Season 2018 Scraped
Season 2019 Scraped
Season 2020 Scraped
Season 2021 Scraped
Season 2022 Scraped
Season 2023 Scraped
Season 2024 Scraped
{'3963': 'Ittihad Alexandria SC', '24297': 'Qanah FC', '8586': 'Dina Farms', '51214': 'Goldi Club', '9094': 'El Masry SC', '7': 'Al Ahly FC', '81628': 'Sekka El Hadid Sohag', '9221': 'Koroum', '664': 'Zamalek SC', '10765': 'Tersana FC', '3369': 'El Mokawloon SC', '13446': 'Ghazl El Mahalla', '3595': 'Ismaily SC', '19466': 'Mansoura SC', '9093': 'Baladiyat El Mahalla', '81629': 'Ghazl Suez', '23471': 'Sohag FC', '12093': 'Harras El Hodoud', '22507': 'Aswan SC', '9218': 'Enppi SC', '9219': "Tala'ea El Gaish", '13448'

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
}

Test = []
Player_Name = []
Player_ID = []
Position = []
Age = []
Another_Team = []
League = []
League_Nationality = []
Cost = []
Nationality = []
In_Out = []
Season = []
Club_Name = []
Club_ID = []
season = 2000

while True:
    for team_id, team_name in Teams_Data.items():
        base_url = f"https://www.transfermarkt.co.uk/el-ahly-kairo/transfers/verein/{team_id}/saison_id/{season}"
        pageTree = requests.get(base_url, headers=headers)
        soup = BeautifulSoup(pageTree.content, 'html.parser')

        # Find all div boxes containing player information
        boxes = soup.find_all('div', class_='box')

        # Iterate over each box (Arrivals or Departures)
        for box in boxes:
            section_name = box.find('h2').get_text(strip=True)  # Get the section name (Arrivals/Departures)
            table = box.find('table', class_='items')  # Get the table with player data
            if table:
                rows = table.find_all('tr', class_=['odd', 'even'])  # Get all rows (players)
                for row in rows:
                    cells = row.find_all('td')  # Get all cells in the row
                    if len(cells) > 10:  # Ensure that the row has enough cells to contain player info
                        # Extract player details
                        name = cells[3].text.strip()
                        Player_Name.append(name)
                        player_id = cells[3].find('a')['href'].split('/')[4]
                        Player_ID.append(player_id)
                        position = cells[4].text.strip()
                        Position.append(position)

                        age = cells[5].text.strip()
                        Age.append(age)

                        another_team = cells[9].text.strip()
                        Another_Team.append(another_team)

                        # Handle league nationality based on another_team value
                        if another_team in ["Without Club", "Retired", "Unknown", "Career break"]:
                            League_Nationality.append(another_team)
                        else:
                            League_Nationality.append("NA")

                        league = cells[10].text.strip()
                        League.append(league)

                        # Extract the cost or fee of the transfer
                        cost = cells[11].text.strip()
                        Cost.append(cost)

                        # Get player nationality (from the image alt or title attribute)
                        img_tag = cells[6].find('img')
                        if img_tag and 'title' in img_tag.attrs:
                            nationality = img_tag['title']
                            Nationality.append(nationality)
                        else:
                            Nationality.append("NA")
                        # Get league nationality (from the image alt or title attribute), only if not already assigned
                        if League_Nationality[-1] == "NA":
                            img_tag2 = cells[10].find('img')
                            if img_tag2 and 'title' in img_tag2.attrs:
                                nationality2 = img_tag2['title']
                                League_Nationality[-1] = nationality2

                        # Append the section name (Arrivals/Departures) to track which section the player belongs to
                        In_Out.append(section_name)
                        Season.append(season)
                        Club_Name.append(team_name)
                        Club_ID.append(team_id)

        # Verify that League_Nationality and Player_Name lists are in sync
        if len(League_Nationality) == len(Player_Name) & len(Nationality) == len(Player_Name):
            print(f'Season {season} for {team_name} Scraped')
        else:
            print(f'Error in Season {season} for {team_name}')

    # Increment season
    season += 1

    # Stop after scraping up to season 2024
    if season > 2024:
        break


Season 2000 for Ittihad Alexandria SC Scraped
Season 2000 for Qanah FC Scraped
Season 2000 for Dina Farms Scraped
Season 2000 for Goldi Club Scraped
Season 2000 for El Masry SC Scraped
Season 2000 for Al Ahly FC Scraped
Season 2000 for Sekka El Hadid Sohag Scraped
Season 2000 for Koroum Scraped
Season 2000 for Zamalek SC Scraped
Season 2000 for Tersana FC Scraped
Season 2000 for El Mokawloon SC Scraped
Season 2000 for Ghazl El Mahalla Scraped
Season 2000 for Ismaily SC Scraped
Season 2000 for Mansoura SC Scraped
Season 2000 for Baladiyat El Mahalla Scraped
Season 2000 for Ghazl Suez Scraped
Season 2000 for Sohag FC Scraped
Season 2000 for Harras El Hodoud Scraped
Season 2000 for Aswan SC Scraped
Season 2000 for Enppi SC Scraped
Season 2000 for Tala'ea El Gaish Scraped
Season 2000 for Suez Cement Scraped
Season 2000 for Asyut Cement Scraped
Season 2000 for Aluminium Nag Hamadi Scraped
Season 2000 for Petrojet Scraped
Season 2000 for Olympic Alexandria Scraped
Season 2000 for Tanta SC Sc

In [4]:
import csv
from itertools import zip_longest
file_list = [Club_Name, Player_Name,Position,Age,Nationality, Another_Team, League, League_Nationality, In_Out, Season ,Cost, Player_ID, Club_ID]
exported = zip_longest(*file_list)
with open ("C:\Users\ABHassan\Desktop\Egyptian League\TransferMarket Web Scraping\Datasets\Transfers Players 2000 to 2024.csv", "w") as myfile:
  wr = csv.writer (myfile)
  wr.writerow(["Club Name", "Player Name","Position","Age","Nationality", "From / To", "League", "Country of League", "In / Out" , "Season", "Cost", "Player ID", "Club ID"])
  wr.writerows(exported)

In [5]:
df = pd.read_csv('C:\Users\ABHassan\Desktop\Egyptian League\TransferMarket Web Scraping\Datasets\Transfers Players 2000 to 2024.csv')
df.head()

,Club Name,Player Name,Position,Age,Nationality,From / To,League,Country of League,In / Out,Season,Cost,Player ID,Club ID
0,Ittihad Alexandria SC,Abu Bakar Bah,Striker,22,Sierra Leone,Tersana,Premier League,Egypt,Arrivals,2000,free transfer,776274,3963
1,Ittihad Alexandria SC,Hassan Mostafa,Defensive Midfield,20,Egypt,El Ahly U21,Egypt,Egypt,Arrivals,2000,?,38174,3963
2,Ittihad Alexandria SC,Mostafa Reyad,Right-Back,34,Egypt,Retired,NaN,Retired,Departures,2000,?,637467,3963
3,Ittihad Alexandria SC,Karam Morsi,Centre-Forward,-,Egypt,Unknown,NaN,Unknown,Departures,2000,?,785906,3963
4,Ittihad Alexandria SC,Magdy El Rashidi,Striker,-,Egypt,Tersana,Premier League,Egypt,Departures,2000,?,787553,3963


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20314 entries, 0 to 20313
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Club Name          20314 non-null  object
 1   Player Name        20314 non-null  object
 2   Position           20314 non-null  object
 3   Age                20314 non-null  object
 4   Nationality        20311 non-null  object
 5   From / To          20314 non-null  object
 6   League             18546 non-null  object
 7   Country of League  20283 non-null  object
 8   In / Out           20314 non-null  object
 9   Season             20314 non-null  int64 
 10  Cost               20301 non-null  object
 11  Player ID          20314 non-null  int64 
 12  Club ID            20314 non-null  int64 
dtypes: int64(3), object(10)
memory usage: 2.0+ MB


In [7]:
df.isnull().sum()

,0
Club Name,0
Player Name,0
Position,0
Age,0
Nationality,3
From / To,0
League,1768
Country of League,31
In / Out,0
Season,0


In [8]:
df_null = df[df['Nationality'].isnull()]
df_null

,Club Name,Player Name,Position,Age,Nationality,From / To,League,Country of League,In / Out,Season,Cost,Player ID,Club ID
1484,Mansoura SC,Ahmed Ezz Elregal,Centre-Back,19,NaN,Al-Hilal B.,Libya,Libya,Departures,2007,?,147561,19466
2871,Zamalek SC,Ahmed Ezz Elregal,Centre-Back,21,NaN,Zamalek U21,Egypt,Egypt,Arrivals,2010,-,147561,664
5000,Zamalek SC,Ahmed Ezz Elregal,Centre-Back,24,NaN,Ljungskile SK,Superettan,Sweden,Departures,2013,?,147561,664


In [9]:
df['Nationality'].fillna('Egypt', inplace=True)

In [10]:
df.isnull().sum()

,0
Club Name,0
Player Name,0
Position,0
Age,0
Nationality,0
From / To,0
League,1768
Country of League,31
In / Out,0
Season,0


In [11]:
df_null = df[df['Country of League'].isnull()]
df_null

,Club Name,Player Name,Position,Age,Nationality,From / To,League,Country of League,In / Out,Season,Cost,Player ID,Club ID
235,Mansoura SC,Amr Zaki,Centre-Forward,18,Egypt,Own Youth,NaN,NaN,Arrivals,2001,?,38178,19466
1168,Mansoura SC,Ibrahim Salah,Defensive Midfield,19,Egypt,Own Youth,NaN,NaN,Arrivals,2006,?,105957,19466
2081,El Sharkia Zagazig,Farag Kamal,Central Midfield,22,Egypt,Own Youth,NaN,NaN,Arrivals,2008,-,729411,39576
2475,Petrojet,Marwan Mohsen,Centre-Forward,20,Egypt,Own Youth,NaN,NaN,Arrivals,2009,?,149532,10957
3061,Tala'ea El Gaish,Islam Mohamed Zaky Sarhan,Attacking Midfield,22,Egypt,Own Youth,NaN,NaN,Arrivals,2010,-,581517,9219
3167,Telecom Egypt Sports Club,Hossam El Sanea,Central Midfield,16,Egypt,Own Youth,NaN,NaN,Arrivals,2010,?,844095,13447
4110,El Nasr,Ahmed Hussein,Right Winger,14,Egypt,Own Youth,NaN,NaN,Arrivals,2011,-,544993,23469
5105,Mansoura SC,Mahmoud Gomaa,Left-Back,22,Egypt,Own Youth,NaN,NaN,Arrivals,2013,-,885234,19466
5106,Mansoura SC,Amir Hegazy,Central Midfield,17,Egypt,Own Youth,NaN,NaN,Arrivals,2013,-,841670,19466
5134,Sohag FC,Mohamed Mazzika,Goalkeeper,19,Egypt,Own Youth,NaN,NaN,Arrivals,2013,-,1159513,23471


In [12]:
df['Country of League'].fillna('Egypt', inplace=True)
df.isnull().sum()

,0
Club Name,0
Player Name,0
Position,0
Age,0
Nationality,0
From / To,0
League,1768
Country of League,0
In / Out,0
Season,0


In [13]:
df_null = df[df['Cost'].isnull()]
df_null

,Club Name,Player Name,Position,Age,Nationality,From / To,League,Country of League,In / Out,Season,Cost,Player ID,Club ID
2689,Wadi Degla FC,Nano,Centre-Back,24,Egypt,Unknown,NaN,Unknown,Arrivals,2009,NaN,69768,18234
3670,Harras El Hodoud,Mohamed Tarek,Left-Back,21,Egypt,Talaea El Gaish,Premier League,Egypt,Arrivals,2011,NaN,210713,12093
3748,Tala'ea El Gaish,Mohamed Tarek,Left-Back,21,Egypt,Harras Hodoud,Premier League,Egypt,Departures,2011,NaN,210713,9219
4592,El Shorta,Mohab Emad,Midfielder,21,Egypt,Unknown,NaN,Unknown,Departures,2012,NaN,159921,18268
6060,Enppi SC,Kareem Tarek,Left Winger,22,Egypt,El Gouna,Premier League,Egypt,Departures,2014,NaN,341346,9218
6229,El Gouna FC,Kareem Tarek,Left Winger,22,Egypt,Enppi SC,Premier League,Egypt,Arrivals,2014,NaN,341346,20572
9700,Petrol Asyut,Ahmed Turk,Centre-Forward,26,Egypt,Unknown,NaN,Unknown,Arrivals,2017,NaN,1165217,16216
10192,Pyramids FC,Nabil Daoudi,Centre-Forward,34,Morocco,AS Salé,Morocco,Morocco,Arrivals,2017,NaN,185511,44664
12242,Tersana FC,Mostafa Naser,Right Winger,28,Egypt,A. Nag Hamadi,Egypt,Egypt,Departures,2019,NaN,469381,10765
12666,Aluminium Nag Hamadi,Mostafa Naser,Right Winger,28,Egypt,Tersana,Egypt,Egypt,Arrivals,2019,NaN,469381,16215


In [14]:
for index, row in df.iterrows():
    if row['In / Out'] == 'Arrivals':
        df.at[index, 'To Team'] = row['Club Name']
        df.at[index, 'From Team'] = row['From / To']
    elif row['In / Out'] == 'Departures':
        df.at[index, 'From Team'] = row['Club Name']
        df.at[index, 'To Team'] = row['From / To']

df.head(20)

,Club Name,Player Name,Position,Age,Nationality,From / To,League,Country of League,In / Out,Season,Cost,Player ID,Club ID,To Team,From Team
0,Ittihad Alexandria SC,Abu Bakar Bah,Striker,22,Sierra Leone,Tersana,Premier League,Egypt,Arrivals,2000,free transfer,776274,3963,Ittihad Alexandria SC,Tersana
1,Ittihad Alexandria SC,Hassan Mostafa,Defensive Midfield,20,Egypt,El Ahly U21,Egypt,Egypt,Arrivals,2000,?,38174,3963,Ittihad Alexandria SC,El Ahly U21
2,Ittihad Alexandria SC,Mostafa Reyad,Right-Back,34,Egypt,Retired,NaN,Retired,Departures,2000,?,637467,3963,Retired,Ittihad Alexandria SC
3,Ittihad Alexandria SC,Karam Morsi,Centre-Forward,-,Egypt,Unknown,NaN,Unknown,Departures,2000,?,785906,3963,Unknown,Ittihad Alexandria SC
4,Ittihad Alexandria SC,Magdy El Rashidi,Striker,-,Egypt,Tersana,Premier League,Egypt,Departures,2000,?,787553,3963,Tersana,Ittihad Alexandria SC
5,Ittihad Alexandria SC,Ahmed El Kass,Second Striker,34,Egypt,Retired,NaN,Retired,Departures,2000,-,101258,3963,Retired,Ittihad Alexandria SC
6,Ittihad Alexandria SC,Amr El Hadidi,Defender,30,Egypt,Retired,NaN,Retired,Departures,2000,-,209816,3963,Retired,Ittihad Alexandria SC
7,Qanah FC,Tarek Fahim,Attacking Midfield,32,Egypt,Ismaily,Premier League,Egypt,Arrivals,2000,?,624492,24297,Qanah FC,Ismaily
8,Qanah FC,Ahmed Ayoub,Sweeper,28,Egypt,El Sharkia,Egypt,Egypt,Arrivals,2000,?,510485,24297,Qanah FC,El Sharkia
9,Qanah FC,Mostafa Kamal,Goalkeeper,26,Egypt,Al Ahly,Premier League,Egypt,Arrivals,2000,?,39611,24297,Qanah FC,Al Ahly


In [15]:
df['Sub Position'] = df['Position']
df.head()

,Club Name,Player Name,Position,Age,Nationality,From / To,League,Country of League,In / Out,Season,Cost,Player ID,Club ID,To Team,From Team,Sub Position
0,Ittihad Alexandria SC,Abu Bakar Bah,Striker,22,Sierra Leone,Tersana,Premier League,Egypt,Arrivals,2000,free transfer,776274,3963,Ittihad Alexandria SC,Tersana,Striker
1,Ittihad Alexandria SC,Hassan Mostafa,Defensive Midfield,20,Egypt,El Ahly U21,Egypt,Egypt,Arrivals,2000,?,38174,3963,Ittihad Alexandria SC,El Ahly U21,Defensive Midfield
2,Ittihad Alexandria SC,Mostafa Reyad,Right-Back,34,Egypt,Retired,NaN,Retired,Departures,2000,?,637467,3963,Retired,Ittihad Alexandria SC,Right-Back
3,Ittihad Alexandria SC,Karam Morsi,Centre-Forward,-,Egypt,Unknown,NaN,Unknown,Departures,2000,?,785906,3963,Unknown,Ittihad Alexandria SC,Centre-Forward
4,Ittihad Alexandria SC,Magdy El Rashidi,Striker,-,Egypt,Tersana,Premier League,Egypt,Departures,2000,?,787553,3963,Tersana,Ittihad Alexandria SC,Striker


In [16]:
df['Position'] = df['Position'].replace('Centre-Back', 'Defender').replace('Right-Back', 'Defender').replace('Left-Back', 'Defender').replace('Sweeper', 'Defender')
df['Position'] = df['Position'].replace('Right-Winger', 'Forward').replace('Left-Winger', 'Forward)').replace('Central Midfield', 'Midfielder').replace('Right Midfield', 'Midfielder').replace('Left Midfield', 'Midfielder').replace('Attacking Midfield', 'Midfielder').replace('Second Striker', 'Forward').replace('Attacking Midfield', 'Midfielder').replace('Attack', 'Forward')
df['Position'] = df['Position'].replace('Midfield' , 'Midfielder').replace('Defensive Midfield' , 'Midfielder').replace('Centre-Forward' , 'Forward').replace('Right Winger' , 'Forward').replace('Left Winger' , 'Forward').replace('Striker', 'Forward')


In [17]:
df.head(10)

,Club Name,Player Name,Position,Age,Nationality,From / To,League,Country of League,In / Out,Season,Cost,Player ID,Club ID,To Team,From Team,Sub Position
0,Ittihad Alexandria SC,Abu Bakar Bah,Forward,22,Sierra Leone,Tersana,Premier League,Egypt,Arrivals,2000,free transfer,776274,3963,Ittihad Alexandria SC,Tersana,Striker
1,Ittihad Alexandria SC,Hassan Mostafa,Midfielder,20,Egypt,El Ahly U21,Egypt,Egypt,Arrivals,2000,?,38174,3963,Ittihad Alexandria SC,El Ahly U21,Defensive Midfield
2,Ittihad Alexandria SC,Mostafa Reyad,Defender,34,Egypt,Retired,NaN,Retired,Departures,2000,?,637467,3963,Retired,Ittihad Alexandria SC,Right-Back
3,Ittihad Alexandria SC,Karam Morsi,Forward,-,Egypt,Unknown,NaN,Unknown,Departures,2000,?,785906,3963,Unknown,Ittihad Alexandria SC,Centre-Forward
4,Ittihad Alexandria SC,Magdy El Rashidi,Forward,-,Egypt,Tersana,Premier League,Egypt,Departures,2000,?,787553,3963,Tersana,Ittihad Alexandria SC,Striker
5,Ittihad Alexandria SC,Ahmed El Kass,Forward,34,Egypt,Retired,NaN,Retired,Departures,2000,-,101258,3963,Retired,Ittihad Alexandria SC,Second Striker
6,Ittihad Alexandria SC,Amr El Hadidi,Defender,30,Egypt,Retired,NaN,Retired,Departures,2000,-,209816,3963,Retired,Ittihad Alexandria SC,Defender
7,Qanah FC,Tarek Fahim,Midfielder,32,Egypt,Ismaily,Premier League,Egypt,Arrivals,2000,?,624492,24297,Qanah FC,Ismaily,Attacking Midfield
8,Qanah FC,Ahmed Ayoub,Defender,28,Egypt,El Sharkia,Egypt,Egypt,Arrivals,2000,?,510485,24297,Qanah FC,El Sharkia,Sweeper
9,Qanah FC,Mostafa Kamal,Goalkeeper,26,Egypt,Al Ahly,Premier League,Egypt,Arrivals,2000,?,39611,24297,Qanah FC,Al Ahly,Goalkeeper


In [18]:
df.to_csv('C:\Users\ABHassan\Desktop\Egyptian League\TransferMarket Web Scraping\Datasets\Transfers Players 2000 to 2024.csv', index=False)